In [1]:
# mount to your drive and access your dataset
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# LOAD DATA

In [2]:
# replace this path to your dataset directory
DATA_ROOT_DIR="/content/drive/MyDrive/Colab/BERT/shopee-sentiment"
!ls $DATA_ROOT_DIR

sample_submission.csv  train.csv	     train_preprocess_unsegment.csv
test.csv	       train.gsheet	     train_preprocess_unsegment.gsheet
test_preprocess.csv    train_preprocess.csv  train.xlsx


In [4]:
# include some dependence
import pandas as pd
import numpy as np
from torch.utils.data import random_split, DataLoader, Dataset
import torch.nn as nn
import torch

DATA_DIR = DATA_ROOT_DIR + '/train.csv'

In [5]:
# Use pandas to read csv, this will return a excel like table data
train = pd.read_csv(DATA_DIR,usecols=['text', 'class']).dropna()
train.head()

,text,class
0,"Đến quán 2 lần thôi , rất là thích !\nQuán tuy...",1
1,Đến quán vào tối chủ_nhật . Có band hát . Khá ...,0
2,Phục_vụ lâu quá mặc_dù khách rất vắng .\nĐợi g...,0
3,"Ko gian bé_tí , quán chật_chội , đông người nê...",0
4,"Khi mình order , đặt bánh thì nhận được sự tiế...",1


In [6]:
X_train = train['text']
y_train = train['class']
X_train

0        Đến quán 2 lần thôi , rất là thích !\nQuán tuy...
1        Đến quán vào tối chủ_nhật . Có band hát . Khá ...
2        Phục_vụ lâu quá mặc_dù khách rất vắng .\nĐợi g...
3        Ko gian bé_tí , quán chật_chội , đông người nê...
4        Khi mình order , đặt bánh thì nhận được sự tiế...
                               ...                        
26995    Không_gian đẹp . Đồ uống bình thg . Cheese ngo...
26996    Chỉ có hai từ thất_vọng mới diễn_tả được cảm_g...
26997    Hôm vào quán có bói bài Tarot , nên cũng bon_c...
26998    Va ̀ o ngô ̀ i đơ ̣ i 20p mơ ́ i đươ ̣ c phu ̣...
26999    Chổ hình_như thấy que quen , hình_như là ở trê...
Name: text, Length: 27000, dtype: object

# Pre-processing

In [7]:
!pip install pyvi
import unicodedata
import regex as re
from pyvi import ViTokenizer, ViPosTagger

     |████████████████████████████████| 8.5 MB 4.7 MB/s 
     |████████████████████████████████| 743 kB 49.3 MB/s 


**Normalize unicode**

In [8]:
def normalize_unicode(text):
	text = unicodedata.normalize('NFC', text)
	return text

**Lower case**

In [9]:
def lower_case(text):
	return text.lower()

**Punctuation**

In [10]:
import gensim
def nom_punctuation(text):  
  text = text.replace('_',' ')
  text = text.replace('\n',' ')
  text = gensim.utils.simple_preprocess(text)
  text = ' '.join(text)
  return text

**Pre-processing 1**

In [11]:
def preprocess(text):
	text = normalize_unicode(text)
	text = lower_case(text)
	text = nom_punctuation(text)
	return text

In [12]:
X_train = X_train.apply(preprocess)
X_train[1]

'đến quán vào tối chủ nhật có band hát khá ổn tuy nhiên không gian tầng hơi hẹp ngồi cả ra vỉa hè đồ uống ko ngon lămz'

**Tokenize**

In [13]:
from pyvi.ViTokenizer import ViTokenizer
import nltk
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

def token(text):
  text = ViTokenizer.tokenize(text)
  text = tokenizer.tokenize(text)
  return text

X_train = X_train.apply(token)
X_train

0        [đến, quán, lần, thôi, rất, là, thích, quán, t...
1        [đến, quán, vào, tối, chủ_nhật, có, band, hát,...
2        [phục_vụ, lâu, quá, mặc_dù, khách, rất, vắng, ...
3        [ko, gian, bé, tí, quán, chật_chội, đông, ngườ...
4        [khi, mình, order, đặt, bánh, thì, nhận, được,...
                               ...                        
26995    [không_gian, đẹp, đồ, uống, bình, thg, cheese,...
26996    [chỉ, có, hai, từ, thất_vọng, mới, diễn_tả, đư...
26997    [hôm, vào, quán, có, bói, bài, tarot, nên, cũn...
26998    [va, ngô, đơ, mơ, đươ, phu, vu, cô, freeze, tr...
26999    [chổ, hình_như, thấy, que, quen, hình_như, là,...
Name: text, Length: 27000, dtype: object

**Remove stopword**

In [16]:
stop_words = open('/content/drive/MyDrive/Colab/BERT/vietnamese-stopwords-dash.txt', 'r')
stop_words = stop_words.readlines()
stop_words = pd.Series(stop_words)
stop_words = stop_words.apply(lambda x: x.replace('\n',''))
def remove_last_space(word):
    word = " ".join(word.split())
    return word
stop_word = stop_words.apply(lambda x: remove_last_space(x))
stop_words = pd.Series.tolist(stop_words)

In [17]:
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stop_words]
    return text

X_train = X_train.apply(lambda x: remove_stopwords(x))
X_train

0        [quán, quán, nằm, hẻm, quán, rộng_rãi, thoáng,...
1        [quán, tối, chủ_nhật, band, hát, ổn, không_gia...
2        [phục_vụ, mặc_dù, vắng, đợi, ly, cà_phê, khéo,...
3        [ko, gian, bé, tí, quán, chật_chội, đông, phục...
4        [order, bánh, tiếp_đãi, tư_vấn, nói_chuyện, gầ...
                               ...                        
26995    [không_gian, đẹp, đồ, uống, bình, thg, cheese,...
26996    [hai, thất_vọng, diễn_tả, cảm_giác, casa, đi, ...
26997    [hôm, quán, bói, tarot, bon_chen, ai_ngờ, trật...
26998    [va, ngô, đơ, mơ, đươ, phu, vu, freeze, tra, x...
26999    [chổ, hình_như, que, quen, hình_như, gò, vấp, ...
Name: text, Length: 27000, dtype: object

In [18]:
X_train = X_train.apply(lambda x: ' '.join(x))
X_train = X_train.apply(lambda x : x.replace('_',' '))
X_train

0        quán quán nằm hẻm quán rộng rãi thoáng mát sạc...
1        quán tối chủ nhật band hát ổn không gian tầng ...
2          phục vụ mặc dù vắng đợi ly cà phê khéo quảng bá
3        ko gian bé tí quán chật chội đông phục vụ đồ u...
4        order bánh tiếp đãi tư vấn nói chuyện gần gũi ...
                               ...                        
26995    không gian đẹp đồ uống bình thg cheese ngon ta...
26996    hai thất vọng diễn tả cảm giác casa đi tô cổng...
26997    hôm quán bói tarot bon chen ai ngờ trật lất kh...
26998    va ngô đơ mơ đươ phu vu freeze tra xanh va coo...
26999    chổ hình như que quen hình như gò vấp đường lê...
Name: text, Length: 27000, dtype: object

In [23]:
X_train[2]

'phục vụ mặc dù vắng đợi ly cà phê khéo quảng bá'

In [20]:
train['text'] = X_train

In [21]:
train

,text,class
0,quán quán nằm hẻm quán rộng rãi thoáng mát sạc...,1
1,quán tối chủ nhật band hát ổn không gian tầng ...,0
2,phục vụ mặc dù vắng đợi ly cà phê khéo quảng bá,0
3,ko gian bé tí quán chật chội đông phục vụ đồ u...,0
4,order bánh tiếp đãi tư vấn nói chuyện gần gũi ...,1
...,...,...
26995,không gian đẹp đồ uống bình thg cheese ngon ta...,0
26996,hai thất vọng diễn tả cảm giác casa đi tô cổng...,0
26997,hôm quán bói tarot bon chen ai ngờ trật lất kh...,1
26998,va ngô đơ mơ đươ phu vu freeze tra xanh va coo...,0


In [22]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Colab/BERT")
train.to_csv('preprocess_data.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
